Библиотеки


In [ ]:
#@title Подключение Google Drive
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt


In [ ]:
TRAIN_DIR = '/content/drive/MyDrive/my_yolov8'   #полный путь к папке сохраним в переменной TRAIN_DIR
os.mkdir(TRAIN_DIR)                              #при помощии команды "mkdir" создадим новую папку




In [ ]:
print(os.listdir('/content/drive/MyDrive/'))

['Google Фото', 'Python_Colab', 'Colab Notebooks', 'Train_results', 'Стаж_Каски_нарезка_090016"', 'Conbine_dataset_3_classes-1_80-20-10-str.zip', 'my_yolov8']


In [ ]:
!pip install gdown

import gdown

url = 'https://drive.google.com/file/d/1XayHB3TuXj9ztSgCNQlONsnDVIQnYjgW/view?usp=sharing'
output = 'Conbine_dataset_3_classes-1_80-20-10-str.zip'

gdown.download(url, output, quiet=False)

/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:35: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=1XayHB3TuXj9ztSgCNQlONsnDVIQnYjgW
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/1XayHB3TuXj9ztSgCNQlONsnDVIQnYjgW/view?usp=sharing
To: /content/Conbine_dataset_3_classes-1_80-20-10-str.zip
85.0kB [00:00, 30.8MB/s]


'Conbine_dataset_3_classes-1_80-20-10-str.zip'

In [ ]:
!ls

Conbine_dataset_3_classes-1_80-20-10-str.zip  drive  sample_data


In [ ]:
!unzip -o /content/drive/MyDrive/Conbine_dataset_3_classes-1_80-20-10-str.zip -d {TRAIN_DIR}


Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: /content/drive/MyDrive/my_yolov8/Conbine_dataset_3_classes-1_80-20-10-str/valid/labels/005858.txt  
  inflating: /content/drive/MyDrive/my_yolov8/Conbine_dataset_3_classes-1_80-20-10-str/valid/labels/006933.txt  
  inflating: /content/drive/MyDrive/my_yolov8/Conbine_dataset_3_classes-1_80-20-10-str/valid/labels/001184.txt  
  inflating: /content/drive/MyDrive/my_yolov8/Conbine_dataset_3_classes-1_80-20-10-str/valid/labels/004597.txt  
  inflating: /content/drive/MyDrive/my_yolov8/Conbine_dataset_3_classes-1_80-20-10-str/valid/labels/003646.txt  
  inflating: /content/drive/MyDrive/my_yolov8/Conbine_dataset_3_classes-1_80-20-10-str/valid/labels/001954.txt  
  inflating: /content/drive/MyDrive/my_yolov8/Conbine_dataset_3_classes-1_80-20-10-str/valid/labels/002689.txt  
  inflating: /content/drive/MyDrive/my_yolov8/Conbine_dataset_3_classes-1_80-20-10-str/valid/labels/001249.txt  
  inflating: /content/drive/

In [ ]:
!ls {TRAIN_DIR}

Conbine_dataset_3_classes-1_80-20-10-str


In [ ]:
# Задание пути к yaml-файлу
data_path = '/content/drive/MyDrive/my_yolov8/Conbine_dataset_3_classes-1_80-20-10-str/data_drive.yaml'
# Просмотр содержимого yaml-файла
!cat {data_path}

names:
- Hardhat
- NO-Hardhat
- Person
nc: 3
roboflow:
  license: CC BY 4.0
  project: conbine_dataset_3_classes
  url: https://universe.roboflow.com/ai-qoi0z/conbine_dataset_3_classes/dataset/1
  version: 1
  workspace: ai-qoi0z
test: /content/drive/MyDrive/my_yolov8/Conbine_dataset_3_classes-1_80-20-10-str/test/images
train: /content/drive/MyDrive/my_yolov8/Conbine_dataset_3_classes-1_80-20-10-str/train/images
val: /content/drive/MyDrive/my_yolov8/Conbine_dataset_3_classes-1_80-20-10-str/valid/images


In [ ]:
# Зададим новый 'TRAIN_DIR'
TRAIN_DIR = '/content/drive/MyDrive/my_yolov8/Conbine_dataset_3_classes-1_80-20-10-str/'
# Выведем количество файлов в рабочих директориях
print(len(os.listdir(TRAIN_DIR+'/train/images/')))
print(len(os.listdir(TRAIN_DIR+'/train/labels/')))
print(len(os.listdir(TRAIN_DIR+'/valid/images/')))
print(len(os.listdir(TRAIN_DIR+'/valid/labels/')))
print(len(os.listdir(TRAIN_DIR+'/test/images/')))
print(len(os.listdir(TRAIN_DIR+'/test/labels/')))


3504
3504
2517
2517
1236
1236


Обучение


In [ ]:
#@title Загружаем yolov8
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO

Ultralytics YOLOv8.1.2 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.1/78.2 GB disk)


In [ ]:
import torch
import gc
torch.cuda.empty_cache()  # Очистить видеопамять
gc.collect()  # Очистить оперативную память


30

In [ ]:
import torch
import gc


model = YOLO('yolov8s.pt')

total_epochs = 200
save_interval = 50
epochs_per_iteration = total_epochs // save_interval  # Количество эпох между сохранениями

for iteration in range(epochs_per_iteration):
    epochs_to_train = min(total_epochs - iteration * save_interval, save_interval)
    model.train(data=data_path, imgsz=640, epochs=epochs_to_train, name='/content/drive/MyDrive/my_yolov8/yolov8s_str')

    current_epoch = (iteration + 1) * save_interval  # Текущая эпоха
    torch.save(model.state_dict(), f'yolov8s_epoch_{current_epoch}.pt')  # Сохранение весов модели
    torch.cuda.empty_cache()  # Очистить видеопамять
    gc.collect()  # Очистить оперативную память

100%|██████████| 21.5M/21.5M [00:00<00:00, 165MB/s]


Ultralytics YOLOv8.1.2 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/my_yolov8/Conbine_dataset_3_classes-1_80-20-10-str/data_drive.yaml, epochs=50, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8s_str, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_c

100%|██████████| 755k/755k [00:00<00:00, 15.9MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 78.8MB/s]


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      47/50      4.23G     0.5814     0.3108     0.8553         30        640: 100%|██████████| 219/219 [01:41<00:00,  2.17it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 79/79 [00:41<00:00,  1.88it/s]

                   all       2517       9054      0.941      0.913       0.95      0.715



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      48/50      4.23G     0.5955     0.3151     0.8509         45        640:  51%|█████     | 112/219 [00:51<00:40,  2.65it/s]